In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
from pathlib import Path

import numpy as np
import yaml
import tensorflow.compat.v1 as tf

# from contact_graspnet.orig.contact_graspnet.data import load_available_input_data
from contact_graspnet.orig.contact_graspnet.visualization_utils import visualize_grasps, show_image

from contact_graspnet.models import ContactGraspnet
from contact_graspnet.dataloading import YCBSimulationData

/home/moritz/Documents/ContactGraspnetBenchmark/contact_graspnet/orig/pointnet2/tf_ops/sampling


In [3]:
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

    tf.disable_eager_execution()
    physical_devices = tf.config.experimental.list_physical_devices("GPU")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
checkpoint_dir = Path.cwd().parent / "contact_graspnet" / "checkpoints" / "scene_test_2048_bs3_hor_sigma_001"
config_path = checkpoint_dir / "config.yaml"

example_data_path = Path.cwd().parent / "contact_graspnet" / "data" / "test_data" / "7.npy"
ycb_data_path = Path.home() / "Documents" / "ycb_sim_data_2"

In [5]:
ycb_sim_datset = YCBSimulationData(ycb_data_path)
ycb_sample = ycb_sim_datset[0]

In [6]:
with open(config_path,'r') as f:
    global_config = yaml.safe_load(f)

global_config['OPTIMIZER']['batch_size'] = 1    


In [7]:
model = ContactGraspnet(global_config, checkpoint_dir)

--- Get model


/home/moritz/Documents/ContactGraspnetBenchmark/contact_graspnet/orig/pointnet2/utils/tf_util.py:571: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  momentum=bn_decay, axis=axis, name=scope,


('loading ', '/home/moritz/Documents/ContactGraspnetBenchmark/contact_graspnet/checkpoints/scene_test_2048_bs3_hor_sigma_001/model.ckpt-54054')


In [8]:
pred_grasps_cam, scores, contact_pts = model(ycb_sample.points)

Generated 4 grasps


In [9]:
visualize_grasps(
    ycb_sample.points, pred_grasps_cam, scores, plot_opencv_cam=True#, pc_colors=pc_colors
)

Visualizing...takes time
